# Integration of CWE ID labels
* Generate new cwe_id label which integrate the cwe id labels of all three dataset

In [47]:
import numpy as np
import pandas as pd
import os

In [48]:
print(os.getcwd())
data_path = '../datasets_'

/home/user/TransVulDet/data_preprocessing


In [49]:
def concat_to_df(df):
    df_temp = []
    for chunk in df:
        df_temp.append(chunk)

    df = pd.concat(df_temp,ignore_index = True)
    return df

In [50]:

msr_df = pd.read_csv(f'{data_path}/MSR.csv', chunksize=10000, index_col=0)

In [51]:
mvd_df = pd.read_csv(f'{data_path}/MVD.csv', chunksize=10000, index_col=0)

In [52]:
cfd_df = pd.read_csv('CVEfixes/CFD_cwe_id.csv', index_col=0)

In [53]:
msr_df = concat_to_df(msr_df)
print(msr_df.shape)
msr_df.head(5)

(145743, 3)


,code,cwe_id,vul
0,static PHP_FUNCTION(preg_match)\n{\n\tphp_do_p...,119,0
1,static PHP_FUNCTION(preg_match_all)\n{\n\tphp_...,119,0
2,static PHP_FUNCTION(preg_replace)\n{\n\tpreg_r...,119,0
3,static PHP_FUNCTION(preg_replace_callback)\n{\...,119,0
4,static PHP_FUNCTION(preg_filter)\n{\n\tpreg_re...,119,0


In [54]:
mvd_df = concat_to_df(mvd_df)
print(mvd_df.shape)
mvd_df.head(5)

(178865, 3)


,code,cwe_id,vul
0,const char *svn_relpath__internal_style(const ...,0,0
1,static void goodG2B() 64\nchar * data ; 66\nch...,0,0
2,static void goodB2G2() 289\nwchar_t * password...,0,0
3,"int avformat_open_input(AVFormatContext **ps,c...",0,0
4,void CWE319_Cleartext_Tx_Sensitive_Info__w32_w...,311,1


In [55]:
print(cfd_df.size)
cfd_df.head(5)

7909


,cwe_id
0,CWE-252
1,NVD-CWE-Other
2,CWE-20
3,CWE-119
4,CWE-19


### Check overlapped dataset

In [56]:
def normalize_cwe(cwe_list):
    normalized_cwe_list = []
    for cwe in cwe_list:
        #split the CWEs that contain multiple entries
        multiple_cwe = cwe.split(',')
        for single_cwe in multiple_cwe:
            single_cwe = single_cwe.strip() #remove leading/trailing whitespaces
            if '-' in single_cwe:
                cwe_prefix, cwe_number = single_cwe.split('-')
                # remove leading zeroes
                cwe_number = str(int(cwe_number))
                normalized_cwe = cwe_prefix + '-' + cwe_number
            else:
                normalized_cwe = single_cwe
            normalized_cwe_list.append(normalized_cwe)
    return list(set(normalized_cwe_list))

In [57]:
msr_cwe_list = msr_df['cwe_id'].unique().tolist()
print(len(msr_cwe_list))
msr_df['cwe_id'].unique()

78


array([ 119,  200,    0,   20,  416,  835,  125,  347,  190,   94,  362,
        704,  354,   78,   59,  269,  617,  476,  787,  754,  502,   22,
        665,  772,  290,  400,  369,  284,   93,  415,  674,  311,  330,
        327,   77,  134,  732,  287,   89,  285,   74,   79,  404,  862,
        834,  494,  755,  129,  191,  682,  522,  918,  358,  295,  770,
        346,  532,  426,  824,  693,  352,  436,   90,  763,  120,  611,
        601,  706,  281, 1021,  668,  664,  172,  909,  209,  345,  331,
        252])

In [58]:
mvd_cwe_list = mvd_df['cwe_id'].unique().tolist()
print(len(mvd_cwe_list))
mvd_df['cwe_id'].unique()

34


array([  0, 311, 706, 704, 666, 119, 404, 400, 369,  74, 506, 362, 662,
       476,  20, 758, 327, 668, 190, 467, 170, 676, 187, 670, 573, 191,
       754, 221, 610, 469, 665, 138, 834, 673])

In [59]:
cfd_cwe_list = cfd_df['cwe_id'].unique().tolist()
print(len(cfd_cwe_list))
# print(cfd_df['cwe_id'].unique())

cfd_df['cwe_id'] = cfd_df['cwe_id'].astype(str)
cfd_df = cfd_df[~cfd_df['cwe_id'].str.contains('NVD-')]
cfd_df = cfd_df[~cfd_df['cwe_id'].str.contains(',')]
cfd_df['cwe_id'] = cfd_df['cwe_id'].str.replace('CWE-', '')
cfd_df['cwe_id'] = cfd_df['cwe_id'].replace('non-vulnerable', 0)
cfd_df['cwe_id'] = cfd_df['cwe_id'].astype(int)

cfd_cwe_list = cfd_df['cwe_id'].unique().tolist()
print(len(cfd_cwe_list))
print(cfd_cwe_list)

209
207
[252, 20, 119, 19, 94, 255, 189, 79, 264, 120, 399, 362, 193, 310, 200, 787, 772, 134, 22, 476, 400, 59, 89, 665, 434, 835, 190, 369, 16, 287, 78, 352, 640, 284, 330, 276, 611, 269, 275, 502, 668, 74, 77, 125, 916, 416, 732, 754, 918, 415, 17, 191, 704, 254, 203, 116, 273, 295, 763, 18, 681, 601, 426, 444, 534, 90, 617, 521, 199, 331, 93, 327, 384, 908, 345, 843, 347, 361, 320, 358, 798, 21, 335, 184, 285, 326, 388, 532, 863, 862, 346, 755, 404, 129, 674, 613, 770, 943, 354, 494, 834, 639, 311, 290, 121, 1188, 338, 522, 682, 1187, 667, 774, 470, 91, 1236, 824, 307, 88, 113, 425, 185, 401, 471, 436, 312, 749, 178, 915, 1321, 670, 306, 672, 776, 1021, 909, 209, 838, 427, 459, 131, 319, 684, 697, 707, 114, 350, 212, 706, 281, 552, 208, 669, 297, 428, 73, 441, 250, 122, 117, 294, 80, 829, 98, 610, 87, 367, 538, 759, 378, 379, 303, 924, 305, 913, 527, 680, 565, 1333, 328, 590, 457, 359, 644, 23, 641, 277, 1336, 786, 840, 805, 788, 825, 823, 321, 922, 126, 662, 791, 279, 324, 27, 288

In [67]:
# List of non-existing CWE IDs
non_exist_cwe_id_list = [16, 17, 18, 19, 21, 189, 199, 254, 255, 264, 275, 310, 320, 361, 388, 399, 534, 769, 840, 1187]

total_cwe_id_list = msr_cwe_list + mvd_cwe_list + cfd_cwe_list
print("Before removing overlapped cwe ids: ",len(total_cwe_id_list))
total_cwe_id_list = sorted(list(set(total_cwe_id_list)))
print("After removing overlapped cwe ids: ",len(total_cwe_id_list))
# Remove values from total_cwe_id_list that exist in non_exist_cwe_id_list
total_cwe_id_list = [cwe_id for cwe_id in total_cwe_id_list if cwe_id not in non_exist_cwe_id_list]
print("After removing non existing cwe ids: ",len(total_cwe_id_list))
print(total_cwe_id_list)

Before removing overlapped cwe ids:  319
After removing overlapped cwe ids:  223
After removing non existing cwe ids:  204
[0, 20, 22, 23, 27, 59, 73, 74, 77, 78, 79, 80, 87, 88, 89, 90, 91, 93, 94, 98, 113, 114, 116, 117, 119, 120, 121, 122, 125, 126, 129, 131, 134, 138, 170, 172, 178, 184, 185, 187, 190, 191, 193, 200, 203, 208, 209, 212, 221, 241, 248, 250, 252, 266, 269, 273, 276, 277, 279, 281, 284, 285, 287, 288, 290, 294, 295, 297, 303, 304, 305, 306, 307, 311, 312, 319, 321, 324, 326, 327, 328, 330, 331, 335, 338, 345, 346, 347, 350, 352, 354, 358, 359, 362, 367, 369, 378, 379, 384, 400, 401, 404, 415, 416, 425, 426, 427, 428, 434, 436, 441, 444, 457, 459, 467, 469, 470, 471, 475, 476, 494, 502, 506, 521, 522, 527, 532, 538, 552, 565, 573, 590, 601, 610, 611, 613, 617, 639, 640, 641, 644, 662, 664, 665, 666, 667, 668, 669, 670, 672, 673, 674, 676, 680, 681, 682, 684, 693, 697, 704, 706, 707, 732, 749, 754, 755, 758, 759, 763, 770, 772, 774, 776, 786, 787, 788, 791, 798, 805, 82

In [68]:
import csv
with open("preprocessed_datasets/total_cwe_id_list.csv", "w") as file:
    writer = csv.writer(file)
    for item in total_cwe_id_list:
        writer.writerow([item])

In [64]:
# def check_match_vul_and_CWE_ID(df,cwe_id_col_name):
#     count = 0
#     for index, row in df.iterrows():
#         if row['vul'] == 0 and row[cwe_id_col_name] != 'non-vulnerable':
#             print("vul and CWE ID are not matched!!!!!",row['vul'], row['cwe_id'])
#             break

#     if count==0:
#         print("All vul and CWE ID are matched correctly")
        
# def check_match_vul_and_label(df):
#     count = 0
   
#     for index, row in df.iterrows():
#         if row['vul'] == 0 and row['label'] != 0:
#             print("vul and label are not matched!!!!!",row['vul'], row['label'])
#             break
#     if count==0:
#         print("All vul and label are matched correctly")


            

In [65]:
# check_match_vul_and_CWE_ID(msr_df,'cwe_id')

KeyError: 'CWE ID'

In [ ]:
# check_match_vul_and_CWE_ID(mvd_df,'cwe_id')

All vul and CWE ID are matched correctly


In [ ]:
# # add new column based on dictionary values
# mvd_df['label'] = mvd_df['cwe_id'].apply(lambda x: total_cwe_dict.get(x))

# mvd_pre_df = mvd_df[['code', 'vul', 'cwe_id','label']]
# mvd_pre_df.head(5)

,code,vul,cwe_id,label
0,const char *svn_relpath__internal_style(const ...,0,non-vulnerable,0
1,static void goodG2B() 64\nchar * data ; 66\nch...,0,non-vulnerable,0
2,static void goodB2G2() 289\nwchar_t * password...,0,non-vulnerable,0
3,"int avformat_open_input(AVFormatContext **ps,c...",0,non-vulnerable,0
4,void CWE319_Cleartext_Tx_Sensitive_Info__w32_w...,1,CWE-311,86


In [ ]:
# # add new column based on dictionary values
# msr_df['label'] = msr_df['cwe_id'].apply(lambda x: total_cwe_dict.get(x))

# msr_pre_df = msr_df[['func_before', 'vul', 'cwe_id','label']]
# msr_pre_df = msr_pre_df.rename(columns={'func_before':'code'})
# msr_pre_df.head(5)

,code,vul,cwe_id,label
0,static bool check_rodc_critical_attribute(stru...,0,non-vulnerable,0
1,static int samldb_add_entry(struct samldb_ctx ...,0,non-vulnerable,0
2,static int samldb_add_entry_callback(struct ld...,0,non-vulnerable,0
3,static int samldb_add_handle_msDS_IntId(struct...,0,non-vulnerable,0
4,static int samldb_add_step(struct samldb_ctx *...,0,non-vulnerable,0


In [ ]:
# check_match_vul_and_label(mvd_pre_df)

All vul and label are matched correctly


In [ ]:
# check_match_vul_and_label(mvd_pre_df)

All vul and label are matched correctly


In [ ]:
# # save it 
# # concatenated_df.to_csv('preprocessed datasets/mvd_msr.csv')

# msr_pre_df.to_csv('preprocessed_datasets/MSR_labeled.csv')
# mvd_pre_df.to_csv('preprocessed_datasets/MVD_labeled.csv')